# MNIST AutoEncoder
Implementation of vanilla (no CNN) AutoEncoder.
#### Losses
* Reconstruction Loss (ie: MSE)

#### References
* [Paper](https://arxiv.org/pdf/1511.05644.pdf)
* https://github.com/neale/Adversarial-Autoencoder
* https://github.com/bfarzin/pytorch_aae
* https://blog.paperspace.com/adversarial-autoencoders-with-pytorch/

In [1]:
import mnist_data_pytorch as data
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from tqdm import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)
print('Pytorch version:', torch.__version__)
# Tensorboard
from torch.utils.tensorboard import SummaryWriter
!rm -rf ./runs
writer = SummaryWriter('./runs/train')

# Metaparameters
num_epochs = 10
latent_size = 50
gen_lr = 0.001

Device: cuda:0
Pytorch version: 1.2.0


#### Define Encoder/Decoder/Discriminator

In [2]:
class AutoEncoder(nn.Module):
    def __init__(self, dimensions=784, latent_size=10):
        super(AutoEncoder, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(dimensions, 256),
            nn.Linear(256, 256),
            nn.Linear(256, latent_size),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, 256),
            nn.Linear(256, 256),
            nn.Linear(256, dimensions),
        )
    def forward(self, x):
        latent = self.encoder(x)
        reconstruct = self.decoder(latent)
        # Make things little more easy to MNIST
        reconstruct = torch.sigmoid(reconstruct)
        return reconstruct, latent
    
# Initialize Networks
autoencoder = AutoEncoder(dimensions=784, latent_size=latent_size).to(device)

#### Initialize Optimizers

In [3]:
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=gen_lr)

#### Train loop

In [4]:
for epoch in tqdm(range(num_epochs)):
    running_loss_discriminator = 0.0
    running_loss_generator = 0.0
    running_loss_reconstruction = 0.0
    # Iterate over the data
    for idx_sample, (inputs, _) in enumerate(data.dataloaders['train']):
        inputs = inputs.to(device)
        inputs = torch.flatten(inputs, start_dim=1, end_dim=-1)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Get latent and reconstructed input
        inputs_reconstruct, z_sample = autoencoder(inputs)
        
        # We can use MSE or Huber Loss
        #reconstruct_loss = F.mse_loss(inputs_reconstruct , inputs)
        reconstruct_loss = F.smooth_l1_loss(inputs_reconstruct , inputs)
        
        # Backprop from reconstruction loss
        reconstruct_loss.backward()
        # Optimizer Step
        optimizer.step()
        
        # Update statistics
        running_loss_reconstruction += reconstruct_loss.item() * inputs.size(0)
    
    # Epoch ends
    epoch_loss_reconstruction = running_loss_reconstruction / len(data.dataloaders['train'].dataset)
    
    # Send results to tensorboard
    writer.add_scalar('train/reconstruction', epoch_loss_reconstruction, epoch)
    
    # Send images to tensorboard
    writer.add_images('train/decoder_images', inputs_reconstruct.view(inputs.size(0),1,28,28), epoch)
    writer.add_images('train/input_images', inputs.view(inputs.size(0),1,28,28), epoch)
    
    # Send latent to tensorboard
    writer.add_histogram('train/latent', z_sample, epoch)
    writer.add_histogram('train/reconstruct_images_h', inputs_reconstruct, epoch)
    writer.add_histogram('train/input_images_h', inputs, epoch)
    

100%|██████████| 10/10 [00:33<00:00,  3.38s/it]
